# Train a model using Watson Studio and deploy it in Watson Machine Learning

This notebook will show how to use your annotated images from Cloud Annotations to train an Object Detection model using a Python Notebook in Watson Studio. After training and testing, some extra steps will show how to deploy this model in Watson Machine Learning as an online API. You can use this API from any application afterwards.

As a suggestion you can use this dataset from Kaggle to test Cloud Annotation and this notebook: https://www.kaggle.com/issaisasank/guns-object-detection

### Specify the credentials for the bucket you used in Cloud Annoations

In [ ]:
# credentials = {
#     'BUCKET': '$$$BUCKET$$$',
#     'IBM_API_KEY_ID': '$$$IBM_API_KEY_ID$$$',
#     'IAM_SERVICE_ID': '$$$IAM_SERVICE_ID$$$',
#     'ENDPOINT': '$$$ENDPOINT$$$',
# }
credentials = {
    'IAM_SERVICE_ID': 'iam-ServiceId-f0afd6e2-22d6-433a-91ce-4d02fab0a8e0',
    'IBM_API_KEY_ID': 'Q5ZIqOmUOt9PB2lOZX4n1RzHUO-E_kYQ3RFhbSsEtjfm',
    'ENDPOINT': 'https://s3.us.cloud-object-storage.appdomain.cloud',
    'IBM_AUTH_ENDPOINT': 'https://iam.cloud.ibm.com/oidc/token',
    'BUCKET': 'guns-object-detection'
}

# Setup

In [ ]:
import os
import shutil

if os.path.exists('tmp') and os.path.isdir('tmp'):
    shutil.rmtree('tmp')

CLOUD_ANNOTATIONS_DATA = os.path.join('tmp', credentials['BUCKET'])

os.makedirs(CLOUD_ANNOTATIONS_DATA, exist_ok=True)

In [ ]:
import json
import ibm_boto3
from ibm_botocore.client import Config, ClientError

def download_file_cos(local_file_name, key): 
    '''
    Wrapper function to download a file from cloud object storage using the
    credential dict provided and loading it into memory
    '''
    cos = ibm_boto3.client("s3",
        ibm_api_key_id=credentials['IBM_API_KEY_ID'],
        ibm_service_instance_id=credentials['IBM_API_KEY_ID'],
        config=Config(signature_version="oauth"),
        endpoint_url=credentials['ENDPOINT']
    )
    try:
        res=cos.download_file(Bucket=credentials['BUCKET'], Key=key, Filename=local_file_name)
    except Exception as e:
        print('Exception', e)
    else:
        print('File Downloaded')
        
def get_annotations(): 
    cos = ibm_boto3.client("s3",
        ibm_api_key_id=credentials['IBM_API_KEY_ID'],
        ibm_service_instance_id=credentials['IBM_API_KEY_ID'],
        config=Config(signature_version="oauth"),
        endpoint_url=credentials['ENDPOINT']
    )
    try:
        return json.loads(cos.get_object(Bucket=credentials['BUCKET'], Key='_annotations.json')['Body'].read())
    except Exception as e:
        print('Exception', e)

In [ ]:
annotations = get_annotations()

download_file_cos(os.path.join(CLOUD_ANNOTATIONS_DATA, '_annotations.json'), '_annotations.json')

for image in annotations['annotations'].keys():
    local_path = os.path.join(CLOUD_ANNOTATIONS_DATA, image)
    download_file_cos(local_path, image)

In [ ]:
NUM_TRAIN_STEPS = 500
MODEL_TYPE = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync_2018_07_18'
CONFIG_TYPE = 'ssd_mobilenet_v1_quantized_300x300_coco14_sync'

import os
CLOUD_ANNOTATIONS_MOUNT = os.path.join('tmp', credentials['BUCKET'])
ANNOTATIONS_JSON_PATH   = os.path.join(CLOUD_ANNOTATIONS_MOUNT, '_annotations.json')

CHECKPOINT_PATH = 'tmp/checkpoint'
OUTPUT_PATH     = 'tmp/output'
EXPORTED_PATH   = 'tmp/exported'
DATA_PATH       = 'tmp/data'

LABEL_MAP_PATH    = os.path.join(DATA_PATH, 'label_map.pbtxt')
TRAIN_RECORD_PATH = os.path.join(DATA_PATH, 'train.record')
VAL_RECORD_PATH   = os.path.join(DATA_PATH, 'val.record')

## Installing dependencies

In the next cell we will install the libraries that will be used. Since we are using an older version of Tensorflow and Numpy, compared to the version that is already installed by default in your environment. We highly suggest creating a custom environment in your Watson Studio project for this notebook, using the following configuration:

``````
# Modify the following content to add a software customization to an environment.
# To remove an existing customization, delete the entire content and click Apply.
# The customizations must follow the format of a conda environment yml file.

# Add conda channels below defaults, indented by two spaces and a hyphen.
channels:
  - defaults

# To add packages through conda or pip, remove the # on the following line.
dependencies:

# Add conda packages here, indented by two spaces and a hyphen.
# Remove the # on the following line and replace sample package name with your package name:

# Add pip packages here, indented by four spaces and a hyphen.
# Remove the # on the following lines and replace sample package name with your package name.
  - pip:
    - numpy==1.19.5
    - tensorflow==1.15.2
``````

Use Python 3.7 and any hardware configuration without CPU that you would like. This notebook was not prepared to support training using GPUs in Watson Studio. Use the next cell to install the other dependencies as normal. After creating the environment you will have to change it using the **Information** tab, on the right side menu.

In [ ]:
import os
import sys
import pathlib

# Clone the tensorflow models repository if it doesn't already exist
if "models" in pathlib.Path.cwd().parts:
    while "models" in pathlib.Path.cwd().parts:
        os.chdir('..')
elif not pathlib.Path('models').exists():
    !git clone --depth 1 https://github.com/cloud-annotations/models

# !pip uninstall Cython -y
# !pip uninstall tf_slim -y
# !pip uninstall opencv-python-headless -y
# !pip uninstall lvis -y
# !pip uninstall pycocotools -y
# !pip uninstall numpy -y
# !pip uninstall tensorflow -y    

# !pip install numpy==1.19.5
# !pip install tensorflow==1.15.2
!pip install Cython
!pip install tf_slim
!pip install opencv-python-headless
!pip install lvis --no-deps
!pip install pycocotools

%cd models/research
!protoc object_detection/protos/*.proto --python_out=.

pwd = os.getcwd()
# we need to set both PYTHONPATH for shell scripts and sys.path for python cells
sys.path.append(pwd)
sys.path.append(os.path.join(pwd, 'slim'))
if 'PYTHONPATH' in os.environ:
    os.environ['PYTHONPATH'] += f':{pwd}:{pwd}/slim'
else:
    os.environ['PYTHONPATH'] = f':{pwd}:{pwd}/slim'
%cd ../..

## Testing Tensorflow

In [ ]:
%cd models/research
!python object_detection/builders/model_builder_tf1_test.py
%cd ../..

# Generate a Label Map

One piece of data the Object Detection API needs is a label map protobuf. The label map associates an integer id to the text representation of the label. The ids are indexed by 1, meaning the first label will have an id of 1 not 0.

Here is an example of what a label map looks like:

````
item {
  id: 1
  name: 'Cat'
}

item {
  id: 2
  name: 'Dog'
}

item {
  id: 3
  name: 'Gold Fish'
}
````


In [ ]:
import os
import json

# Get a list of labels from the annotations.json
labels = {}
with open(ANNOTATIONS_JSON_PATH) as f:
    annotations = json.load(f)
    labels = annotations['labels']

# Create a file named label_map.pbtxt
os.makedirs(DATA_PATH, exist_ok=True)
with open(LABEL_MAP_PATH, 'w') as f:
  # Loop through all of the labels and write each label to the file with an id
  for idx, label in enumerate(labels):
    f.write('item {\n')
    f.write("\tname: '{}'\n".format(label))
    f.write('\tid: {}\n'.format(idx + 1)) # indexes must start at 1
    f.write('}\n')

# Generate TFRecords

The TensorFlow Object Detection API expects our data to be in the format of TFRecords.

The TFRecord format is a collection of serialized feature dicts, one for each image, looking something like this:

````
{
  'image/height': 1800,
  'image/width': 2400,
  'image/filename': 'image1.jpg',
  'image/source_id': 'image1.jpg',
  'image/encoded': ACTUAL_ENCODED_IMAGE_DATA_AS_BYTES,
  'image/format': 'jpeg',
  'image/object/bbox/xmin': [0.7255949630314233, 0.8845598428835489],
  'image/object/bbox/xmax': [0.9695875693160814, 1.0000000000000000],
  'image/object/bbox/ymin': [0.5820120073891626, 0.1829972290640394],
  'image/object/bbox/ymax': [1.0000000000000000, 0.9662484605911330],
  'image/object/class/text': (['Cat', 'Dog']),
  'image/object/class/label': ([1, 2])
}
````


In [ ]:
import os
import io
import json
import random

import PIL.Image
import tensorflow as tf

from object_detection.utils import dataset_util
from object_detection.utils import label_map_util

def create_tf_record(images, annotations, label_map, image_path, output):
  # Create a train.record TFRecord file.
  with tf.python_io.TFRecordWriter(output) as writer:
    # Loop through all the training examples.
    for image_name in images:
        try:
            # Make sure the image is actually a file
            img_path = os.path.join(image_path, image_name)   
            if not os.path.isfile(img_path):
                  continue

            # Read in the image.
            with tf.gfile.GFile(img_path, 'rb') as fid:
                  encoded_jpg = fid.read()

            # Open the image with PIL so we can check that it's a jpeg and get the image
            # dimensions.
            encoded_jpg_io = io.BytesIO(encoded_jpg)
            image = PIL.Image.open(encoded_jpg_io)
            if image.format != 'JPEG':
                  raise ValueError('Image format not JPEG')

            width, height = image.size

            # Initialize all the arrays.
            xmins = []
            xmaxs = []
            ymins = []
            ymaxs = []
            classes_text = []
            classes = []

            # The class text is the label name and the class is the id. If there are 3
            # cats in the image and 1 dog, it may look something like this:
            # classes_text = ['Cat', 'Cat', 'Dog', 'Cat']
            # classes      = [  1  ,   1  ,   2  ,   1  ]

            # For each image, loop through all the annotations and append their values.
            for a in annotations[image_name]:
                if ("x" in a and "x2" in a and "y" in a and "y2" in a):
                    label = a['label']
                    xmins.append(a["x"])
                    xmaxs.append(a["x2"])
                    ymins.append(a["y"])
                    ymaxs.append(a["y2"])
                    classes_text.append(label.encode("utf8"))
                    classes.append(label_map[label])

            # Create the TFExample.
            tf_example = tf.train.Example(features=tf.train.Features(feature={
              'image/height': dataset_util.int64_feature(height),
              'image/width': dataset_util.int64_feature(width),
              'image/filename': dataset_util.bytes_feature(image_name.encode('utf8')),
              'image/source_id': dataset_util.bytes_feature(image_name.encode('utf8')),
              'image/encoded': dataset_util.bytes_feature(encoded_jpg),
              'image/format': dataset_util.bytes_feature('jpeg'.encode('utf8')),
              'image/object/bbox/xmin': dataset_util.float_list_feature(xmins),
              'image/object/bbox/xmax': dataset_util.float_list_feature(xmaxs),
              'image/object/bbox/ymin': dataset_util.float_list_feature(ymins),
              'image/object/bbox/ymax': dataset_util.float_list_feature(ymaxs),
              'image/object/class/text': dataset_util.bytes_list_feature(classes_text),
              'image/object/class/label': dataset_util.int64_list_feature(classes),
            }))
            if tf_example:
                # Write the TFExample to the TFRecord.
                writer.write(tf_example.SerializeToString())
        except ValueError:
            print('Invalid example, ignoring.')
            pass
        except IOError:
            print("Can't read example, ignoring.")
            pass

with open(ANNOTATIONS_JSON_PATH) as f:
    annotations = json.load(f)['annotations']
    image_files = [image for image in annotations.keys()]
    # Load the label map we created.
    label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)

    random.seed(42)
    random.shuffle(image_files)
    num_train = int(0.7 * len(image_files))
    train_examples = image_files[:num_train]
    val_examples = image_files[num_train:]

    create_tf_record(train_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, TRAIN_RECORD_PATH)
    create_tf_record(val_examples, annotations, label_map, CLOUD_ANNOTATIONS_MOUNT, VAL_RECORD_PATH)

# Download a base model

Training a model from scratch can take days and tons of data. We can mitigate this by using a pretrained model checkpoint. Instead of starting from nothing, we can add to what was already learned with our own data.

There are several pretrained model checkpoints that can be downloaded from the model zoo.

The model we will be training is the SSD MobileNet architecture. SSD MobileNet models have a very small file size and can execute very quickly, compromising little accuracy, which makes it perfect for running in the browser. Additionally, we will be using quantization. When we say the model is quantized it means instead of using float32 as the datatype of our numbers we are using float16 or int8.

````
float32(PI) = 3.1415927 32 bits
float16(PI) = 3.14 16 bits
int8(PI) = 3 8 bits
````

We do this because it can cut our model size down by around a factor of 4! An unquantized version of SSD MobileNet that I trained was 22.3 MB, but the quantized version was 5.7 MB that's a ~75% reduction 🎉

In [ ]:
import os
import tarfile

import six.moves.urllib as urllib

download_base = 'http://download.tensorflow.org/models/object_detection/'
model = MODEL_TYPE + '.tar.gz'
tmp = 'tmp/checkpoint.tar.gz'

if not (os.path.exists(CHECKPOINT_PATH)):
    # Download the checkpoint
    opener = urllib.request.URLopener()
    opener.retrieve(download_base + model, tmp)

    # Extract all the `model.ckpt` files.
    with tarfile.open(tmp) as tar:
        for member in tar.getmembers():
            member.name = os.path.basename(member.name)
            if 'model.ckpt' in member.name:
                tar.extract(member, path=CHECKPOINT_PATH)

    os.remove(tmp)

# Model Config

The final thing we need to do is inject our pipline with the amount of labels we have and where to find the label map, TFRecord and model checkpoint. We also need to change the the batch size, because the default batch size of 128 is too large for Colab to handle.

In [ ]:

#from google.protobuf import text_format

from object_detection.utils import config_util
from object_detection.utils import label_map_util

pipeline_skeleton = 'models/research/object_detection/samples/configs/' + CONFIG_TYPE + '.config'
configs = config_util.get_configs_from_pipeline_file(pipeline_skeleton)

label_map = label_map_util.get_label_map_dict(LABEL_MAP_PATH)
num_classes = len(label_map.keys())
meta_arch = configs["model"].WhichOneof("model")

override_dict = {
  'model.{}.num_classes'.format(meta_arch): num_classes,
  'train_config.batch_size': 24,
  'train_input_path': TRAIN_RECORD_PATH,
  'eval_input_path': VAL_RECORD_PATH,
  'train_config.fine_tune_checkpoint': os.path.join(CHECKPOINT_PATH, 'model.ckpt'),
  'label_map_path': LABEL_MAP_PATH
}

configs = config_util.merge_external_params_with_configs(configs, kwargs_dict=override_dict)
pipeline_config = config_util.create_pipeline_proto_from_configs(configs)
config_util.save_pipeline_config(pipeline_config, DATA_PATH)

# Start training

We can start a training run by calling the model_main script, passing:

- The location of the pipepline.config we created
- Where we want to save the model
- How many steps we want to train the model (the longer you train, the more potential there is to learn)
- The number of evaluation steps (or how often to test the model) gives us an idea of how well the model is doing

In [ ]:
!rm -rf $OUTPUT_PATH
!python -m object_detection.model_main \
    --pipeline_config_path=$DATA_PATH/pipeline.config \
    --model_dir=$OUTPUT_PATH \
    --num_train_steps=$NUM_TRAIN_STEPS \
    --num_eval_steps=100

# Export inference graph

After your model has been trained, you might have a few checkpoints available. A checkpoint is usually emitted every 500 training steps. Each checkpoint is a snapshot of your model at that point in training. In the event that a long running training process crashes, you can pick up at the last checkpoint instead of starting from scratch.

We need to export a checkpoint to a TensorFlow graph proto in order to actually use it. We use regex to find the checkpoint with the highest training step and export it.

In [ ]:
import os
import re
import json

from object_detection.utils.label_map_util import get_label_map_dict

regex = re.compile(r"model\.ckpt-([0-9]+)\.index")
numbers = [int(regex.search(f).group(1)) for f in os.listdir(OUTPUT_PATH) if regex.search(f)]
TRAINED_CHECKPOINT_PREFIX = os.path.join(OUTPUT_PATH, 'model.ckpt-{}'.format(max(numbers)))

print(f'Using {TRAINED_CHECKPOINT_PREFIX}')

!rm -rf $EXPORTED_PATH
!python -m object_detection.export_inference_graph \
    --pipeline_config_path=$DATA_PATH/pipeline.config \
    --trained_checkpoint_prefix=$TRAINED_CHECKPOINT_PREFIX \
    --output_directory=$EXPORTED_PATH

label_map = get_label_map_dict(LABEL_MAP_PATH)
label_array = [k for k in sorted(label_map, key=label_map.get)]

with open(os.path.join(EXPORTED_PATH, 'labels.json'), 'w') as f:
    json.dump(label_array, f)

# Evaluating the results

In the next steps we will use the images from the evaluation set to **visualize** the results of our model. If you don't see any boxes in your images, consider raising the amount of training steps in the **SETUP** section or adding more training images.

In [ ]:

import os
import numpy as np
from matplotlib import pyplot as plt
from PIL import Image as PImage
from object_detection.utils import visualization_utils as vis_util
from object_detection.utils import label_map_util

# Load the labels
category_index = label_map_util.create_category_index_from_labelmap(LABEL_MAP_PATH, use_display_name=True)

# Load the model
path_to_frozen_graph = os.path.join(EXPORTED_PATH, 'frozen_inference_graph.pb')
detection_graph = tf.Graph()
with detection_graph.as_default():
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(path_to_frozen_graph, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='')

In [ ]:
bbox_images = []
for image_x in val_examples:
    img_path = os.path.join(CLOUD_ANNOTATIONS_MOUNT, image_x)   
    with detection_graph.as_default():
        with tf.Session(graph=detection_graph) as sess:
            # Definite input and output Tensors for detection_graph
            image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
            # Each box represents a part of the image where a particular object was detected.
            detection_boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
            # Each score represent how level of confidence for each of the objects.
            # Score is shown on the result image, together with the class label.
            detection_scores = detection_graph.get_tensor_by_name('detection_scores:0')
            detection_classes = detection_graph.get_tensor_by_name('detection_classes:0')
            num_detections = detection_graph.get_tensor_by_name('num_detections:0')
            image = PImage.open(img_path)
            # the array based representation of the image will be used later in order to prepare the
            # result image with boxes and labels on it.
            (im_width, im_height) = image.size
            image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)
            # Expand dimensions since the model expects images to have shape: [1, None, None, 3]
            image_np_expanded = np.expand_dims(image_np, axis=0)
            # Actual detection.
            (boxes, scores, classes, num) = sess.run(
                [detection_boxes, detection_scores, detection_classes, num_detections],
                feed_dict={image_tensor: image_np_expanded})
            # Visualization of the results of a detection.
            vis_util.visualize_boxes_and_labels_on_image_array(
                image_np,
                np.squeeze(boxes),
                np.squeeze(classes).astype(np.int32),
                np.squeeze(scores),
                category_index,
                use_normalized_coordinates=True,
                line_thickness=8)
            
            bbox_images.append(image_np)

In [ ]:
%matplotlib inline

fig = plt.figure(figsize=(50, 50))  # width, height in inches

for i,bbox_image in enumerate(bbox_images):
    sub = fig.add_subplot(len(bbox_images)+1, 1, i + 1)
    sub.imshow(bbox_image, interpolation='nearest')

### Here you can choose different images from the array to see it in more detail

In [ ]:
%matplotlib inline
plt.figure(figsize=(12, 8))
plt.imshow(bbox_images[6])

# Deploying your model in Watson Machine Leaning

In the following steps we will export the artifacts that were created to a .tar file and upload the model to Watson Machine Learning. Than we will generate an online deployment using this model.

You will need a Watson Machine Leaning instance and an IAM API Key in IBM Cloud that has access to this instance. See the steps in the documentation:

https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-authentication.html

Also, in the new version of WML you will need a Deployment Space and it's ID

https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/ml-spaces_local.html?audience=wdp

In [ ]:
!ls $EXPORTED_PATH/saved_model

In [ ]:
!tar -zcvf guns-object-detection-model.tar.gz -C $EXPORTED_PATH/saved_model .

In [ ]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                   "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"<apikey>"
                  }

client = APIClient(wml_credentials)

In [ ]:
client.set.default_space("<deployment-space-id>")

In [ ]:
client.software_specifications.list()

In [ ]:
model_spec = client.software_specifications.get_id_by_name('tensorflow_1.15-py3.6')

In [ ]:
model_meta = {
    client.repository.ModelMetaNames.NAME              : "Tensorflow Guns Object Detection Model",
    client.repository.ModelMetaNames.DESCRIPTION       : "Guns Object Detection using Kaggle Dataset",
    client.repository.ModelMetaNames.TYPE              : "tensorflow_1.15",
    client.repository.ModelMetaNames.SOFTWARE_SPEC_UID : model_spec
}
model_details_dir = client.repository.store_model( model="guns-object-detection-model.tar.gz", meta_props=model_meta )

In [ ]:
model_id_dir = model_details_dir["metadata"]['id']

In [ ]:
client.hardware_specifications.list()

In [ ]:
meta_props = {
    client.deployments.ConfigurationMetaNames.NAME: "Tensorflow Guns Object Detection Deployment",
    client.deployments.ConfigurationMetaNames.ONLINE: {},
    client.deployments.ConfigurationMetaNames.HARDWARE_SPEC : { "id":  "cf70f086-916d-4684-91a7-264c49c6d425"}
}
deployment_details_dir = client.deployments.create(model_id_dir, meta_props )

In [ ]:
deployment_id = deployment_details_dir['metadata']['id']

# Test the deployed model

Choose one of the images from the evaluation set to score the model using the newly created API. This step can be done in another notebook or custom code, since your deployed model is not dependent of this kernel. 

In [ ]:
img_path = os.path.join(CLOUD_ANNOTATIONS_MOUNT, val_examples[5])   
if os.path.isfile(img_path):
    print("OK")

In [ ]:
image = PImage.open(img_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
(im_width, im_height) = image.size
image_np = np.array(image.getdata()).reshape((im_height, im_width, 3)).astype(np.uint8)

In [ ]:
data = image_np.tolist()
payload_scoring = {
  "input_data": [{
    "values": [data]
  }]
}

In [ ]:
%%time
predictions = client.deployments.score(deployment_id, payload_scoring)

In [ ]:
for x in predictions['predictions']:
    if x['id'] == 'detection_scores':
        scores = x['values'][0]
    if x['id'] == 'detection_boxes':
        boxes = x['values'][0]
    if x['id'] == 'num_detections':
        num = x['values'][0]
    if x['id'] == 'detection_classes':
        classes = x['values'][0]

In [ ]:
vis_util.visualize_boxes_and_labels_on_image_array(
    image_np,
    np.squeeze(boxes),
    np.squeeze(classes).astype(np.int32),
    np.squeeze(scores),
    category_index,
    use_normalized_coordinates=True,
    line_thickness=8)

In [ ]:
%matplotlib inline
plt.figure(figsize=(12, 8))
plt.imshow(image_np)